<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Анализ-данных" data-toc-modified-id="Анализ-данных-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Анализ данных</a></span></li><li><span><a href="#Очиcтка-и-лемматизация" data-toc-modified-id="Очиcтка-и-лемматизация-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Очиcтка и лемматизация</a></span></li><li><span><a href="#Подготовка-признаков" data-toc-modified-id="Подготовка-признаков-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка признаков</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Catboost" data-toc-modified-id="Catboost-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Catboost</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Необходимо обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок.

Необходимо построить модель со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

from sklearn.model_selection import GridSearchCV

from sklearn.metrics import f1_score
from datetime import datetime
import warnings

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ilyan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ilyan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ilyan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
warnings.filterwarnings("ignore")

### Анализ данных

In [3]:
df = pd.read_csv(r'C:\Users\ilyan\projects\Git\Projects\Toxic_comments\toxic_comments.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


In [4]:
display(df['toxic'].unique())
zero = df[df['toxic']==0]['toxic'].count()
total = df.shape[0]
display(f'Positive: {zero}, Share:{zero/total:0.2%}, Negative{total-zero}, Share: {1-zero/total:0.2%}')

array([0, 1], dtype=int64)

'Positive: 143346, Share:89.83%, Negative16225, Share: 10.17%'

Негативных сообщений меньше позитивных в 9 раз. При классификации будет полезно учесть этот дисбаланс.

### Очиcтка и лемматизация

Функция очистки текста от лишних символов

In [5]:
def clear_text(text):
    s = re.sub(r"[^a-zA-Z']", ' ', text)
    return " ".join (s.split())

Функция лемматизации

In [6]:
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])

Применим функции и токенизируем

In [7]:
%%time
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
df['text_lemmatized'] = df['text'].apply(lemmatize_text)
df['text_lemmatized'] = df['text_lemmatized'].apply(clear_text)
df.head()

CPU times: total: 2min 40s
Wall time: 2min 40s


,text,toxic,text_lemmatized
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits made under my userna...
1,D'aww! He matches this background colour I'm s...,0,D'aww He match this background colour I'm seem...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I'm really not trying to edit war It's...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I can't make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,You sir are my hero Any chance you remember wh...


### Подготовка признаков

In [8]:
%%time
target = df['toxic']
corpus = df['text_lemmatized'].values

corpus_train, corpus_test, target_train, target_test = train_test_split(corpus,target, test_size=0.20, random_state=42)

CPU times: total: 46.9 ms
Wall time: 22 ms


Векторизуем корпус

In [9]:
%%time

stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
features_train = count_tf_idf.fit_transform(corpus_train)
features_test = count_tf_idf.transform(corpus_test)

CPU times: total: 57.6 s
Wall time: 57.6 s


## Обучение

### Логистическая регрессия

In [10]:
model = LogisticRegression(class_weight = 'balanced', random_state=42)

parameters = {
    'penalty' : ['l1','l2'], 
    'C'       : np.logspace(-3,3,20),
    'solver'  : ['newton-cg', 'lbfgs', 'liblinear']
}
CV = 5
    
start_time = datetime.now()
lr_GS = GridSearchCV(model, parameters, scoring = 'f1', cv=CV)
lr_GS.fit(features_train,target_train)
lr_GS_time_fit = datetime.now() - start_time

print('Time elapsed (hh:mm:ss.ms) {}'.format(lr_GS_time_fit))
print(" Results from Grid Search " )
print("\n The best score across ALL searched params:\n", lr_GS.best_score_)
print("\n The best parameters across ALL searched params:\n", lr_GS.best_params_)

Time elapsed (hh:mm:ss.ms) 0:15:43.103789
 Results from Grid Search 

 The best score across ALL searched params:
 0.7633855893150552

 The best parameters across ALL searched params:
 {'C': 2.976351441631316, 'penalty': 'l1', 'solver': 'liblinear'}


In [11]:
predictions = lr_GS.best_estimator_.predict(features_test)

print('F1 =', f1_score(target_test, predictions))

F1 = 0.7691216120334895


### Catboost

Сделаем выборку, чтобы модель обучалась приемлемое время.

In [12]:
df_sample = df.sample(30000, random_state = 42).reset_index(drop=True) 

Подготовим признаки для выборки

In [13]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
df_sample['text_lemmatized'] = df_sample['text'].apply(lemmatize_text)
df_sample['text_lemmatized'] = df_sample['text_lemmatized'].apply(clear_text)
df_sample.shape

(30000, 3)

In [14]:
target_sample = df_sample['toxic']
corpus_sample = df_sample['text_lemmatized'].values

corpus_train_sample, corpus_val_sample, target_train_sample, target_val_sample = train_test_split(corpus_sample,target_sample, test_size=0.20, random_state=42)
corpus_valid_sample, corpus_test_sample, target_valid_sample, target_test_sample = train_test_split(corpus_val_sample,target_val_sample, test_size=0.5, random_state=42)

In [15]:
%%time

stopwords = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stopwords)
features_train_sample = count_tf_idf.fit_transform(corpus_train_sample)
features_valid_sample = count_tf_idf.transform(corpus_valid_sample)
features_test_sample = count_tf_idf.transform(corpus_test_sample)

CPU times: total: 10.7 s
Wall time: 10.7 s


In [16]:
features_train_sample.shape

(24000, 55073)

In [17]:
%%time
clf = CatBoostClassifier(random_state=42,n_estimators=80, depth =11)
clf.fit(features_train_sample,target_train_sample, eval_set=(features_valid_sample,target_valid_sample), verbose=False)
predictions_cb = clf.predict(features_test_sample)
f1_result = f1_score(target_test_sample, predictions_cb)
print('F1 =', f1_result)

F1 = 0.6666666666666666
CPU times: total: 1h 45min 24s
Wall time: 12min 16s


## Выводы

Модель логистической регрессии достигла значения 0,76, при этом  быстро обучилась на всем датафрейме. Catboost обучен на 24000 строк, что составляет 15% от всего датафрейма, с результатом 0,66. Вероятно, при обучении на большем объеме данных, модель достигла бы порога 0,75. 